In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sympy import false
from sympy.abc import alpha
from tensorflow.python.ops.linalg.linear_operator_algebra import inverse

from datasetLoader import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from inverse import fit_linear

from tools import model_tester
from tools.reportParser import find_nearest
%matplotlib qt

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [35]:
pore_widths = np.load("data/initial kernels/new_kernel/pore_sizes.npy")
pressures = np.load("data/initial kernels/new_kernel/pressure.npy")[:-14]
kernel = np.load("data/initial kernels/new_kernel/new_kernel.npy")[:, :-14]
with open("data/initial kernels/new_kernel/new_kernel.npy", 'rb') as f:
    data_sorb = np.load(f)[:, :-14]

x, y = load_dataset('data/datasets/carbon_random_normal_micro.npz', cut_end_idx=-14)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)

x_exp, y_exp = load_dataset('data/datasets/exp.npz', cut_end_idx=-14)

x_train_exp, x_test_exp, y_train_exp, y_test_exp = train_test_split(x_exp, y_exp, test_size=0.15, random_state=1)

In [36]:
figure, axis = plt.subplots(3, 4)
for i in range(3):
    for j in range(4):
        k = np.random.randint(0, len(x_train_exp))
        axis[i, j].plot(pressures[:], x_train_exp[k], marker=".")
        #axis[i, j].set_xscale("log")
        ############################
        start_i = 0
        for l in range(len(x_train_exp[k])):
            if x_train_exp[k][l] != 0:
                start_i = l
                break

        end_i = len(x_train_exp[k])
        for l in range(len(x_train_exp[k])-1, -1, -1):
            if x_train_exp[k][l] != 1:
                end_i = l+1
                break

        pd_test = fit_linear(x_train_exp[k][start_i:end_i], kernel[:, start_i:end_i], 0).x
        tw = axis[i, j].twiny()
        tw.plot(pore_widths, pd_test*100, color="r", marker=".")

        # dist = fit_linear(adsorption=isotherm_data[i][start_i:end_i], kernel=kernel[start_i:end_i], alpha=0).x
        axis[i, j].plot(pressures[start_i:end_i], np.dot(pd_test, kernel)[start_i:end_i], marker=".")
        ##############################
        axis[i, j].grid()
plt.show()

In [37]:
figure, axis = plt.subplots(3, 4)
for i in range(3):
    for j in range(4):
        k = np.random.randint(0, len(y_train_exp))
        axis[i, j].plot(pore_widths, y_train_exp[k], marker=".")
        axis[i, j].grid()
plt.show()

In [38]:
figure, axis = plt.subplots(3, 4)
for i in range(3):
    for j in range(4):
        k = np.random.randint(0, len(x_train))
        axis[i, j].plot(pressures, x_train[k], marker=".")
        axis[i, j].grid()
plt.show()

In [42]:
figure, axis = plt.subplots(3, 4)
for i in range(3):
    for j in range(4):
        k = np.random.randint(0, len(x_train))
        axis[i, j].plot(pore_widths, y_train[k], marker=".")
        axis[i, j].grid()
plt.show()

In [41]:
plt.plot(pore_widths, sum(y_train), marker=".")

In [9]:
i = np.random.randint(0, len(x_train))
plt.plot(pressures[:-10], x_train[i], marker=".")
plt.grid()
plt.show()

ValueError: x and y must have same first dimension, but have shapes (171,) and (181,)

In [10]:
class IsothermDataset(Dataset):
    def __init__(self, isotherms, transform=None):
        self.data = torch.tensor(isotherms, dtype=torch.float32).to(device)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        if self.transform:
            x = self.transform(x)
        return x, x

x_mixed_train = np.concatenate((x_train_exp, x_train))
x_mixed_test = np.concatenate((x_test_exp, x_test))

# dataset = IsothermDataset(np.concatenate((x_train_exp, x_train_exp)))
# dataset_test = IsothermDataset(np.concatenate((x_test_exp, x_test_exp)))
dataset = IsothermDataset(np.concatenate((x_mixed_train, x_mixed_train)))
dataset_test = IsothermDataset(np.concatenate((x_mixed_test, x_mixed_test)))


batch_size = 512
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [11]:
y_exp[0].size

117

In [14]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon, z

input_dim = len(x_mixed_train[0])
latent_dim = 16
epochs = 200
learning_rate = 1e-3

model = Autoencoder(input_dim=input_dim, latent_dim=latent_dim)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

def train_autoencoder(model, loader, loader_test):
    model.train()
    total_loss = 0
    total_vloss = 0
    for x, _ in loader:
        optimizer.zero_grad()
        x_recon, _ = model(x)
        loss = criterion(x_recon, x)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    with torch.no_grad():
        for x, _ in loader_test:
            x_recon, _  = model(x)
            vloss = criterion(x_recon, x)
            total_vloss += vloss.item()

    return total_loss / len(loader.dataset), total_vloss / len(loader_test.dataset)


# sample_z = model.encoder(torch.tensor(isotherms_np[0], dtype=torch.float32))

In [15]:
for epoch in range(1, epochs+1):
    loss, vloss = train_autoencoder(model, loader,loader_test)
    if epoch % 1 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss*100:.8f} Test loss: {vloss*100:.8f}")

Epoch 1/200, Loss: 0.00214626 Test loss: 0.00014176
Epoch 2/200, Loss: 0.00009178 Test loss: 0.00006789
Epoch 3/200, Loss: 0.00005339 Test loss: 0.00004523
Epoch 4/200, Loss: 0.00003668 Test loss: 0.00003404
Epoch 5/200, Loss: 0.00002900 Test loss: 0.00002643
Epoch 6/200, Loss: 0.00002595 Test loss: 0.00002659
Epoch 7/200, Loss: 0.00002441 Test loss: 0.00002443
Epoch 8/200, Loss: 0.00002161 Test loss: 0.00001952
Epoch 9/200, Loss: 0.00002043 Test loss: 0.00001754
Epoch 10/200, Loss: 0.00001905 Test loss: 0.00002589
Epoch 11/200, Loss: 0.00001761 Test loss: 0.00001458
Epoch 12/200, Loss: 0.00001622 Test loss: 0.00001336
Epoch 13/200, Loss: 0.00001539 Test loss: 0.00001349
Epoch 14/200, Loss: 0.00001387 Test loss: 0.00001287
Epoch 15/200, Loss: 0.00001321 Test loss: 0.00001159
Epoch 16/200, Loss: 0.00001286 Test loss: 0.00001113
Epoch 17/200, Loss: 0.00001192 Test loss: 0.00001194
Epoch 18/200, Loss: 0.00001136 Test loss: 0.00000892
Epoch 19/200, Loss: 0.00001065 Test loss: 0.00002256
Ep

KeyboardInterrupt: 

In [16]:
torch.save(model, "data/models/torch/autoencoder_exp.pt")

In [17]:
model = torch.load("data/models/torch/autoencoder_exp.pt", weights_only=False)
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=181, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=16, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=181, bias=True)
  )
)

In [48]:
model.eval()
latent_vectors_train = model.encoder(torch.tensor(x_train, dtype=torch.float32).to(device)).detach().cpu().numpy()
latent_vectors_test = model.encoder(torch.tensor(x_test, dtype=torch.float32).to(device)).detach().cpu().numpy()
latent_vectors_test_exp = model.encoder(torch.tensor(x_test_exp, dtype=torch.float32).to(device)).detach().cpu().numpy()

In [49]:
decoded = model.decoder(model.encoder(torch.tensor(x_test_exp, dtype=torch.float32).to(device))).detach().cpu().numpy()

In [50]:
figure, axis = plt.subplots(3, 3)
for i in range(3):
    for j in range(3):
        k=np.random.randint(0, len(decoded))
        axis[i, j].plot(pressures[:], x_test_exp[k], marker=".", label = "origin")
        axis[i, j].plot(pressures[:], decoded[k], marker=".", label = "decoded")
        axis[i, j].grid(True)
axis[i, j].legend()
plt.show()


# k=np.random.randint(0, len(decoded))
# plt.plot(pressures[:-10], x_test_exp[k], marker=".", label = "origin")
# plt.plot(pressures[:-10], decoded[k], marker=".", label = "decoded")
# plt.legend()
# plt.grid(True)
# plt.show()

In [14]:
np.random.seed(0)
labels = None 

pca = PCA(n_components=2)
latent_pca = pca.fit_transform(latent_vectors_train[:100])
latent_pca_exp = pca.fit_transform(latent_vectors_test[:100])

tsne = TSNE(n_components=2, init='pca', random_state=0)
latent_tsne = tsne.fit_transform(latent_vectors_train[:100])
latent_tsne_exp = tsne.fit_transform(latent_vectors_test[:100])

plt.figure()
plt.scatter(latent_pca[:, 0], latent_pca[:, 1], label="train")
plt.scatter(latent_pca_exp[:, 0], latent_pca_exp[:, 1], label="exp")
plt.title("PCA of Latent Space")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

plt.figure()
plt.scatter(latent_tsne[:, 0], latent_tsne[:, 1], label="train")
plt.scatter(latent_tsne_exp[:, 0], latent_tsne_exp[:, 1], label="exp")
for i in range(latent_tsne_exp.shape[0]):
        plt.text(latent_tsne_exp[i, 0], latent_tsne_exp[i, 1], str(i), fontsize=8, ha='center', va='center')
plt.title("t-SNE of Latent Space")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.legend()
plt.show()

C:\Users\anton\PycharmProjects\isotherm\.venv\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
C:\Users\anton\PycharmProjects\isotherm\.venv\lib\site-packages\sklearn\manifold\_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(
C:\Users\anton\PycharmProjects\isotherm\.venv\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
C:\Users\anton\PycharmProjects\isotherm\.venv\lib\site-packages\sklearn\manifold\_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


In [21]:
def find_exact_idx_last(array, value):
    flag = False
    for i in range(len(array)):
        if array[i] == value:
            flag = True
        if flag and array[i] != value:
            return i
def find_alpha(x_exp):
    def triangle_method(log_residuals, log_solution_norms):
        points = np.column_stack((log_residuals, log_solution_norms))
        A = points[0]
        B = points[-1]
        AB = B - A
        AB_norm = np.linalg.norm(AB)
        dist = []
        for i in range(0, len(points)):
            P = points[i]
            AP = P - A
            dist.append(np.linalg.norm(np.cross(AB, AP)) / AB_norm)
    
        return dist
    def calculate_roughness(psd, ord=2):
        return np.linalg.norm(psd, ord=ord)

    alpha_list = np.logspace(-3, 4, 40)
    error_lst = []
    roughness_lst = []
    restored_isotherms = []
    for alpha in alpha_list:
        start_idx = find_exact_idx_last(x_exp, 0)
        y = fit_linear(x_exp[start_idx:], kernel=data_sorb[:, start_idx:], alpha=alpha).x
        restored_isotherm = np.dot(y, data_sorb)
        restored_isotherms.append(restored_isotherm)
        error_lst.append(np.linalg.norm((x_exp[start_idx:] - restored_isotherm[start_idx:]), ord=2))
        roughness_lst.append(calculate_roughness(y))

    dist = triangle_method(np.log(error_lst), np.log(roughness_lst))
    alpha = alpha_list[np.argmax(dist)]
    return alpha



def plot_preds(x, y, preds): 
    NX, NY = 3, 4
    figure, axis = plt.subplots(NX, NY)
    for i in range(NX):
        for j in range(NY):
            
            low_p = False
            while low_p == False:
                k = np.random.randint(0, len(preds))
                for l in range(len(x[k])):
                    if x[k][l]!=0:
                        if pressures[l] < 1e-2:
                            low_p = True
                        break
            
            iso_axis = axis[i, j].twinx().twiny()
            iso_axis.set_xlabel("P/P$^0$",fontsize=8)
            iso_axis.plot(pressures[:], x[k], label="Isotherm", color = 'green')
            kernel = (data_sorb.T[:])
            iso_axis.plot(pressures[:], np.dot(kernel, preds[k][:]), label="Isotherm by model", color="red")
            axis[i, j].set_title(f"№ {k}")
            axis[i, j].title.set_size(10)
            axis[i, j].grid()
            axis[i, j].set_xlabel("nm",fontsize=8)
            axis[i, j].plot(pore_widths, (preds[k]), marker=".", label=f"Model PSD")
            #axis[i, j].plot(pore_widths, y[k], marker=".", label="PSD")
            alpha = find_alpha(x[k])
            start_idx = find_exact_idx_last(x[k], 0)
            L_curve = fit_linear(x[k][start_idx:], kernel=data_sorb[:, start_idx:], alpha=alpha).x
            axis[i, j].plot(pore_widths, L_curve, marker=".", label="L_curve")
            iso_axis.plot(pressures[:], np.dot(kernel, L_curve), label="Isotherm by L_curve", color="yellow")
    plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.9)
    plt.legend()
    axis[0, 0].legend()
    plt.show()

In [22]:
from tools import model_tester
from inverse import fit_linear

error_lst, roughness_lst = model_tester.test_model_predictions(preds, x_test_exp, kernel=data_sorb[:, :-10])
kde_x, kde_error, kde_fun = model_tester.calculate_kde_data(error_lst, stop=150)
print("average error:", np.mean(error_lst))
plt.plot(kde_x, kde_error, label=model_name)
plt.grid(True)
plt.legend()
plt.plot()

NameError: name 'preds' is not defined

In [43]:
class DynamicWeightAveraging:
    def __init__(self, num_tasks, T=2.0):
        self.num_tasks = num_tasks
        self.T = T
        self.loss_history = []  # список списков: [ [L1_1, L1_2, ...], [L2_1, L2_2, ...], ... ]

    def update_weights(self):
        if len(self.loss_history[0]) < 2:
            return np.ones(self.num_tasks) / self.num_tasks

        r = []
        for i in range(self.num_tasks):
            li = self.loss_history[i]
            r_i = li[-1] / (li[-2] + 1e-8)
            r.append(r_i)

        r = np.array(r)
        weights = self.T * np.exp(r / self.T)
        weights /= weights.sum()
        return weights

    def append_losses(self, losses):  # losses — список текущих значений потерь [L1, L2, ...]
        if not self.loss_history:
            self.loss_history = [[] for _ in range(len(losses))]
        for i, l in enumerate(losses):
            self.loss_history[i].append(l)
dwa = DynamicWeightAveraging(num_tasks=2)

In [44]:
dwa.update_weights()

IndexError: list index out of range

In [45]:
x_exp[0].size, y_exp[0].size

(181, 117)

In [51]:
class PSD_model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PSD_model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, output_dim),
            nn.ReLU()
        )

    def forward(self, x):
        psd = self.model(x)
        return psd

class Isotherm_PSD_Dataset(Dataset):
    def __init__(self, x, y, original_x, transform=None):
        self.x = torch.tensor(x, dtype=torch.float32).to(device)
        self.y = torch.tensor(y, dtype=torch.float32).to(device)
        self.original_x = torch.tensor(original_x, dtype=torch.float32).to(device)
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        original_x = self.original_x[idx]
        if self.transform:
            x = self.transform(x)
        return x, y, original_x

train_PSD = Isotherm_PSD_Dataset(latent_vectors_train, (y_train*100), x_train)
test_PSD = Isotherm_PSD_Dataset(latent_vectors_test, (y_test*100), x_test)

batch_size = 512
PSD_loader = DataLoader(train_PSD, batch_size=batch_size, shuffle=True)
PSD_loader_test = DataLoader(test_PSD, batch_size=batch_size, shuffle=False)


model_PSD = PSD_model(input_dim=latent_dim, output_dim=117)
model_PSD.to(device)

optimizer = optim.Adam(model_PSD.parameters(), lr=learning_rate)
criterion = nn.MSELoss()


with open("data/initial kernels/new_kernel/kernel.npy", 'rb') as f:
    data_sorb_torch = torch.tensor(np.load(f))
    data_sorb_torch = data_sorb_torch.to(torch.float32).to(device)

def isoterm_loss(predicted_y, x):
    restored_isotherm = torch.matmul(predicted_y, data_sorb_torch)
    loss = torch.mean((x - restored_isotherm) ** 2)
    return loss

def train_PSD_model(model, loader, loader_test):
    model.train()
    total_loss = 0
    total_vloss = 0
    for x, y, original_x in loader:
        optimizer.zero_grad()
        y_recon = model(x)
        loss = criterion(y_recon, y)
        # iso_loss = isoterm_loss(y_recon, original_x)
        # dwa.append_losses([loss.item(), iso_loss.item()])
        # weights = dwa.update_weights()
        # loss = weights[0] * loss + weights[1] * iso_loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    with torch.no_grad():
        for x, y, original_x in loader_test:
            y_recon  = model(x)
            vloss = criterion(y_recon, y)
            # iso_loss = isoterm_loss(y_recon, original_x)
            # vloss = weights[0] * vloss + weights[1] * iso_loss
            total_vloss += vloss.item()

    return total_loss / len(loader.dataset), total_vloss / len(loader_test.dataset)


In [52]:
epochs = 50
loss_lst = []
vloss_lst = []
for epoch in range(1, epochs+1):
    loss, vloss = train_PSD_model(model_PSD, PSD_loader, PSD_loader_test)
    loss_lst.append(loss)
    vloss_lst.append(vloss)
    if epoch % 1 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss*100:.8f} Test loss: {vloss*100:.8f}")

Epoch 1/50, Loss: 0.00022675 Test loss: 0.00013410
Epoch 2/50, Loss: 0.00013061 Test loss: 0.00010035
Epoch 3/50, Loss: 0.00010495 Test loss: 0.00008780
Epoch 4/50, Loss: 0.00009240 Test loss: 0.00007332
Epoch 5/50, Loss: 0.00008509 Test loss: 0.00007281
Epoch 6/50, Loss: 0.00008120 Test loss: 0.00007144
Epoch 7/50, Loss: 0.00007907 Test loss: 0.00006850
Epoch 8/50, Loss: 0.00007690 Test loss: 0.00006757
Epoch 9/50, Loss: 0.00007533 Test loss: 0.00006877
Epoch 10/50, Loss: 0.00007485 Test loss: 0.00007159
Epoch 11/50, Loss: 0.00007425 Test loss: 0.00006708
Epoch 12/50, Loss: 0.00007243 Test loss: 0.00006410
Epoch 13/50, Loss: 0.00007156 Test loss: 0.00006629
Epoch 14/50, Loss: 0.00007063 Test loss: 0.00006547
Epoch 15/50, Loss: 0.00006826 Test loss: 0.00006294
Epoch 16/50, Loss: 0.00006695 Test loss: 0.00006622
Epoch 17/50, Loss: 0.00006729 Test loss: 0.00006344
Epoch 18/50, Loss: 0.00006575 Test loss: 0.00006420
Epoch 19/50, Loss: 0.00006531 Test loss: 0.00006320
Epoch 20/50, Loss: 0.

In [53]:
plt.plot(loss_lst)
plt.plot(vloss_lst)
plt.show()

In [54]:
model_name = "autoencoder_regressor_micro"
torch.save(model_PSD, f"data/models/torch/{model_name}") 

In [56]:
model_name = "autoencoder_regressor_micro"
model_PSD = torch.load(f"data/models/torch/{model_name}", weights_only=False)

In [57]:
model_PSD.eval()
y_train_PSD = model_PSD.model(torch.tensor(latent_vectors_train, dtype=torch.float32).to(device)).detach().cpu().numpy()
y_test_PSD = model_PSD.model(torch.tensor(latent_vectors_test, dtype=torch.float32).to(device)).detach().cpu().numpy()
y_test_exp_PSD = model_PSD.model(torch.tensor(latent_vectors_test_exp, dtype=torch.float32).to(device)).detach().cpu().numpy()

In [58]:
plot_preds(x_test_exp, y_test_exp, y_test_exp_PSD/100)

In [100]:
np.savez(f"data/models/metrics/{model_name}", x=x_test_exp, y=y_test_exp_PSD)
model_name


'autoencoder_regressor'

In [72]:
model2 = "autoencoder_regressor_combined"
model2_data = np.load(f"data/models/metrics/{model2}.npz")
model2_x = model2_data["x"]
model2_y = model2_data["y"]

In [73]:

model2_y_math = [fit_linear(model2_x[i], data_sorb[:, :-10], 0).x for i in range(len(model2_x))]

In [155]:
plot_preds(model2_x, model2_y_math, model2_y)

NameError: name 'model2_x' is not defined

In [3]:
import pandas as pd
data = pd.read_csv("data/initial kernels/excel/kernel_N2_77K.csv")

In [4]:
pressures = data["# P/P0\\H[nm]"]

In [5]:
a_array = []
for i in (data.columns[1:]).to_numpy():
    a_array.append(float(i))
a_array = np.array(a_array)

In [6]:
new_kernel = data.to_numpy()

In [16]:
np.save("data/initial kernels/new_kernel/pressure.npy", pressures)
np.save("data/initial kernels/new_kernel/pore_sizes.npy", a_array)
np.save("data/initial kernels/new_kernel/kernel.npy", new_kernel[:, 1:].T)

In [15]:
new_kernel[:, 1:]

array([[3.95232682e+00, 3.73193260e+00, 3.32136295e+00, ...,
        1.99026908e-04, 1.76855432e-04, 1.59128603e-04],
       [4.19957099e+00, 3.94315055e+00, 3.55339145e+00, ...,
        2.29265468e-04, 2.03725435e-04, 1.83305333e-04],
       [4.42334610e+00, 4.19271427e+00, 3.78090471e+00, ...,
        2.88698360e-04, 2.56537540e-04, 2.30823899e-04],
       ...,
       [1.53910509e+01, 1.49431184e+01, 1.45971719e+01, ...,
        1.72948254e+01, 1.72948254e+01, 1.72948254e+01],
       [1.53992906e+01, 1.49613487e+01, 1.45868859e+01, ...,
        1.73099488e+01, 1.73099488e+01, 1.73099488e+01],
       [1.53999701e+01, 1.49585595e+01, 1.45743595e+01, ...,
        1.73115993e+01, 1.73115993e+01, 1.73115993e+01]])